**Please run the below code to install the correct packages for the Model to work.**

In [1]:
!pip install igraph
!pip install numpy
!pip install matplotlib

import igraph as ig
import random as rd
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets.embed import embed_minimal_html, dependency_state

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

import warnings
warnings.filterwarnings('ignore')

### Parameter Input:   
The user defined parameters of the model:   
   
**1. Population size**   
This is the number of nodes within the model in which one node represents one person that is suseptable to infection.   
**2. Number of Edges**   
The number of edges for each node enables the transmission of COVID-19 from node to node.   
**3. Number of Days Infected**     
The number of days a node is in the infected state before going to the recovered state. This simulates the number of days the infected person is able to transmit the virus to other people   
**4. Days**   
The total number of days the model runs for. 
   
Once the paramters have been chosen by the user, click **Run Model Button.**    
   
**5. Enter Filename to Save Plot**   
This parameter enables the user to save the plot generated by the model under any name they chose for example Graph 1. This allows the user to save and easily identify mutliple different plots to the working directory on binder as image files (png).   
   
Once the user has input the file name, click **Save Results button.** The plot is saved under the filename defined in Enter Filename to Save Plot parameter and found in the working directory in binder in the window to the left. 
   
### The Barabasi-Albert Model    
The Barabasi-Albert Alogrithm is used to generate a scale-free network. The process begins with two arbitary nodes with 1 edge ($m_{0}$) as your starting point. From that point on, the algorithm adds one new node to the pre-existing nodes at random (m($\ge$$m_{0}$) until the model is created. Each node is able to attach to any node in the system. However, the number of edges a node has is set by the parameter; number of interactions (Bertotti and Modanese et al., 2019).  
Once built, one node in the model is randomly chosen to go from the susceptible state (S) to infected state (I).   
Since each infected node can contaminate neighbouring nodes, the duration of infection is increased with each node changing to the infected state (probs_inf). When the duration of infection is equal to the number of days of infection, the node can change from the infected state to the recovered state. The maximum degree of each infected node, representing the maximum number of interactions between each node/person, is recorded in max_deg and printed.    
This simulates the infectivity rates as people go from asymptomatic to symptomatic states, according to the findings by He et al., in Wuhan, China. They suggested that infectiousness for COVID-19 started at day 2-3 days before showing intial symptoms, peaking at 15 hours before symptoms were shown. Once symptoms were shown the decline infectivity began steadily due to the immune response. They found that infectivity lasted between 2-14 days (He et al., 2020). 
The cluster coefficient is then calculated, measuring how dense the network calculated is. The higher the cluster coefficient is, the tighter the cluster of nodes are and the denser the network. Lower cluster coefficients suggests that the nodes are more linear and the network is less dense (Masuda et al., 2018).

In [3]:
#Sliders
layout = widgets.Layout(width='auto', height='40px')

pop_slider = widgets.IntSlider(value = 100, min=0, max=1000, step=1, description="Population Size: ", 
                               style={'description_width': 'initial', 'width': '800px'},
                               disabled=False, continuous_update=False, orientation="horizontal", readout=True, 
                               readout_format="d") 
embed_minimal_html("population_size.html", views=pop_slider, title = "population size widget export")
display(pop_slider)
node_slider = widgets.IntSlider(value = 10, min=0, max=500, step=1, description="Number of Edges: ",
                                style={'description_width': 'initial', 'width': '800px'},
                                disabled=False, continuous_update=False, orientation="horizontal", readout=True, 
                                readout_format="d")
embed_minimal_html("node.html", views=node_slider, title = "node widget export")
display(node_slider)
infect_len_slider = widgets.IntSlider(value = 5, min=30, max=365, step=1, description="Number of Days Infected: ", 
                                      style={'description_width': 'initial', 'width': '800px'}, 
                                      disabled=False, continuous_update=False, orientation="horizontal", readout=True, 
                                      readout_format="d")
embed_minimal_html("infection_len.html", views=infect_len_slider, title = "infection length size widget export")
display(infect_len_slider)
days_slider = widgets.IntSlider(value = 120, min=10, max=500, step=1, description="Days: ", 
                                      style={'description_width': 'initial', 'width': '800px'}, 
                                      disabled=False, continuous_update=False, orientation="horizontal", readout=True, 
                                      readout_format="d")
embed_minimal_html("days.html", views=infect_len_slider, title = "days widget export")
display(days_slider)


#Buttons
button = widgets.Button(description='Run Model')
sub_button = widgets.Button(description='Save Results')
out = widgets.Output()
fig_size = (40, 10)

#Run Model Button
def on_button_clicked(_):
      with out:
        #clearing previous outputs
        out.clear_output()
        plt.close()
        #sliders
        pop=pop_slider.value
        print("Population Size:", pop)
        edge_per_node = node_slider.value
        print("Number of Edges: ", edge_per_node)
        infect_len = infect_len_slider.value
        print("Number of days of infection: ", infect_len)
        days = days_slider.value
        print("Days: ", days)
        #Model
        font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 18,
        }
        params = {'legend.fontsize': 16,'legend.handlelength': 2.}
        plt.rcParams.update(params)
        color_dict = {"S": "blue", "I": "red", "R": "green"}
        g = ig.Graph.Barabasi(pop, edge_per_node,power=1)
        print("Total number of interactions within the model:", len(g.es))
        g.vs["state"] = "S"
        g.vs["duration"] = 0
        i = rd.randint(0, pop-1)
        g.vs[i]["state"] = "I"
        nb_S = [pop]
        nb_I = [1]
        nb_R = [0]
        max_deg_I = [g.vs[i].degree()]
        print("Max degree of each infected node: ", max_deg_I)
        tot_deg_I = [g.vs[i].degree()]
        probs_inf = [0.01,0.01,0.1,0.2,0.3,0.3,0.3,0.25,0.2,0.15,0.1,0.05,0.01,0.01]
        print("Probability of Infection: ", len(probs_inf))
        time_1perc = [] #time when the first 1% is infected
        time_1hub = [] #time when the first hub is infected
        deg_cutoff = 8
        for time in range(120): #no. of days
            if len(g.vs.select(state_eq = "I"))> pop*0.01  and len(time_1perc) is 0:
                time_1perc.append(time)
            for n in g.vs.select(state_eq = "I"): #iterates through each node in the network
                if g.vs[n.index]["duration"] is 0 and len(g.neighbors(n)) > deg_cutoff and len(time_1hub) is 0:
                    time_1hub.append(time)
                g.vs[n.index]["duration"] += 1 #from day 0 to infect_len this node continues to infect
                day_inf = g.vs[n.index]["duration"]
                for nb in g.neighbors(n): #iterates through neighbours of that node
                    if g.vs[nb]["state"] == "S": #if node is infected...
                        r = rd.random() #random state
                        #if r < p_infect: #random state says to infect according to p_infect
                        if r < probs_inf[day_inf]:
                            g.vs[nb]["state"] = "I" #change state to infected
                #if g.vs[n.index]["duration"] >= infect_len: #after infect_len that node changes to recovered
                if g.vs[n.index]["duration"] >= rd.randrange(1,14):
                    g.vs[n.index]["state"] = "R"
            nb_S.append(len(g.vs.select(state_eq = "S"))) #no. of susceptibles in population
            nb_I.append(len(g.vs.select(state_eq = "I"))) #no. of infecteds in population
            nb_R.append(len(g.vs.select(state_eq = "R"))) #no. of recovereds in population
            if len(g.vs.select(state_eq = "I")) > 0: #if there are infecteds
                max_deg_I.append(max(g.vs.select(state_eq = "I").degree())) #records max no. of neighbours within infecteds
                tot_deg_I.append(sum(g.vs.select(state_eq = "I").degree())) #records total no. of neighbours within infecteds
            else:
                max_deg_I.append(0)  
        #Results
        cl_coeff = g.transitivity_undirected()
        print("Cluster coefficient:", cl_coeff)
        f = plt.figure(figsize=fig_size)
        f.add_subplot(1, 2, 1)
        plt.plot(nb_S, label='S')
        plt.plot(nb_I, label='I')
        plt.plot(nb_R, label='R')
        plt.xlabel("Days")
        plt.ylabel("Number of nodes")
        plt.title("The rates of suseptiblity, infection and recovery during a given amount of time within a population")
        plt.legend()
        fig = plt.gcf()
        plt.show()
        return(fig)
        
        
button.on_click(on_button_clicked)

#text widget to input filname you want to save the graph
text = widgets.Text(value='Graph 1', placeholder='Filename', description='Enter Filename to Save Plot:', 
                    disabled=False, layout=layout, style= {'description_width': 'initial'})
display(text)

#Save Results Button
def save_button(_):
    out.clear_output()
    save_fig = on_button_clicked(_)
    save_fig.savefig(text.value)
    plt.close()
    
sub_button.on_click(save_button)

buttons = widgets.VBox([button, sub_button])
widgets.VBox([buttons, out])

IntSlider(value=100, continuous_update=False, description='Population Size: ', max=1000, style=SliderStyle(des…

IntSlider(value=10, continuous_update=False, description='Number of Edges: ', max=50, style=SliderStyle(descri…

IntSlider(value=30, continuous_update=False, description='Number of Days Infected: ', max=365, min=30, style=S…

IntSlider(value=120, continuous_update=False, description='Days: ', max=500, min=10, style=SliderStyle(descrip…

Text(value='Graph 1', description='Enter Filename to Save Plot:', layout=Layout(height='40px', width='auto'), …

### References   
Bertotti M.L and Modanese G (2019) The Configuration Model for Barabasi-Albert Network. *Applied Network Science*, **4**, Page 32   
He X, Lau EH, Wu P, Deng W et al., (2020) Temporal Dynamics in Viral Shedding and Transmissibility in COVID-19. *Nature Medicine*, **26**, Pages 672-675   
Masuda N, Sakaki M, Ezaki T and Watanabe T (2018) Clustering Coefficients for Correlation Networks. *Frontiers in Neuroinformatics*, **12**, Page 7